In [2]:
import cv2, dlib  # 이미지 표현 및 눈 좌표 얻어오기
import numpy as np  # 데이터 처리
from imutils import face_utils  # 얼굴 분석
from tensorflow import keras  # 모델 학습 및 테스트
from playsound import playsound  # 소리 재생
import threading  # 스레드 사용

# 추출할 눈 이미지 사이즈
IMG_SIZE = (34, 26)
# 눈을 감은 프레임을 세줄 변수
n_count = 0
# 경보음 다중재생 방지 변수
global is_playing
is_playing = False

# 얼굴에 68개의 점을 찍음 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 학습한 모델을 불러옴
model = keras.models.load_model('models/2018_12_17_22_58_35.h5')
model.summary()



# 눈을 찾아주는 함수
def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect


def playSound():
    global is_playing
    t = threading.Thread(target=thread)
    # 스레드를 시작한후 join해서 동작이 멈추길 기다린 후 is_plaing을 false로 바꿔 소리 재생이 가능하도록.
    t.start()
    t.join()
    is_playing = False


def thread():
    playsound("sound.wav")


# main
#cap = cv2.VideoCapture('videos/5.mp4')
cap = cv2.VideoCapture('C:\\eGovFrame-3.10.0\\bin\\eclipse\\checkupload\\membervid\\2\\vid_2022_12_05_09_28_10.mp4')

#cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, img_ori = cap.read()

    if not ret:
        break

    img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.5, fy=0.5)

    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)

        # cv2.imshow('l', eye_img_l)
        # cv2.imshow('r', eye_img_r)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        if pred_l < 0.1 and pred_r < 0.1:
            n_count += 1
        else:
            n_count = 0
            is_playing = False
            # playsound("")

        if n_count > 100:
            cv2.putText(img, "Wake up", (120, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            # 스레드에서 사운드 재생
            if not is_playing:
                is_playing = True
                t = threading.Thread(target=playSound)
                t.start()

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

        state_l = state_l % pred_l
        state_r = state_r % pred_r

        # 색 지정
        if pred_l > 0.1:
            l_color = (255, 255, 255)
        else:
            l_color = (0, 0, 255)
        if pred_r > 0.1:
            r_color = (255, 255, 255)
        else:
            r_color = (0, 0, 255)

        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(l_color), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(r_color), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (l_color), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (r_color), 2)

    # 크기 조절
    img = cv2.resize(img, dsize=(480, 320), interpolation=cv2.INTER_LINEAR)
    cv2.imshow('result', img)
    if cv2.waitKey(1) == ord('q'):
        break

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 17, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 8, 64)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 6, 8, 128)         7385

In [21]:
import requests

def gotojsp2():
    URL = "http://localhost:8085/controller/flask/upsleepy?deepresult="+file_path
    response = requests.get(URL)
    
file_path = "22"

gotojsp2()

In [ ]:
#플라스크 실행
from flask import Flask #간단히 플라스크 서버를 만든다

import urllib.request

app = Flask(__name__)

from flask import Flask

app = Flask(__name__)

import requests
    
@app.route("/gotojsp3")
def gotojsp3():
    URL = "http://localhost:8085/controller/flask/upsleepy?deepresult=23"
    response = requests.get(URL)
 
    return 'gotojsp3'

    
    
if __name__ == '__main__':
    app.run(debug=False,host="127.0.0.1",port=5000)





 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2022 11:37:46] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [06/Dec/2022 11:37:50] "GET /gotojsp3 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 11:37:57] "GET /gotojsp3 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 11:37:57] "GET /gotojsp3 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 11:37:59] "GET /gotojsp3 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 11:37:59] "GET /gotojsp3 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 11:37:59] "GET /gotojsp3 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 11:38:14] "GET /gotojsp3 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 11:38:14] "GET /gotojsp3 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 11:38:14] "GET /gotojsp3 HTTP/1.1" 200 -


In [5]:
import requests



URL = "http://localhost:8085/controller/flask/upsleepy?deepresult=딥런이결과"
response = requests.get(URL)

In [ ]:
# 환경 셋팅
import h5py
# import necessary packages
import cvlib as cv
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import load_model
from PIL import ImageFont, ImageDraw, Image

import cv2, dlib  # 이미지 표현 및 눈 좌표 얻어오기
import numpy as np  # 데이터 처리
from imutils import face_utils  # 얼굴 분석
from tensorflow import keras  # 모델 학습 및 테스트
from playsound import playsound  # 소리 재생
import threading  # 스레드 사용


#도난방지
car_500 = load_model('theaf_modelh5/car_500.h5')
car_500.summary()


import os # 파일 마지막꺼 출력
import asyncio#비동기 함수 사용
import requests#통신 관련


#파일 업로드 처리,플라스크 관련
from flask import Flask, render_template, request, redirect, url_for
from flaskext.mysql import MySQL
from werkzeug.utils import secure_filename
import urllib.request

# 추출할 눈 이미지 사이즈
IMG_SIZE = (34, 26)
# 눈을 감은 프레임을 세줄 변수
#n_count = 0
# 경보음 다중재생 방지 변수
global is_playing
is_playing = False


n_count=0 #도난방지 확인 프레임 버퍼
    

# open webcam
webcam = cv2.VideoCapture('checkc_upload_vid_thef/65/test_vid.mp4')

new_count = 0
new_fps=1


# loop through frames
while webcam.isOpened():

    new_count+=1
    # read frame from webcam 
    status, frame = webcam.read()

    if not status:
        print("Could not read frame")
        exit()

    if(new_count%new_fps==0): 

        # apply face detection
        face, confidence = cv.detect_face(frame)

        # loop through detected faces
        for idx, f in enumerate(face):

            (startX, startY) = f[0], f[1]
            (endX, endY) = f[2], f[3]

            if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:

                face_region = frame[startY:endY, startX:endX]

                face_region1 = cv2.resize(face_region, (224, 224), interpolation = cv2.INTER_AREA)

                x = img_to_array(face_region1)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)

                prediction = car_500.predict(x)

                if prediction < 0.7: # 마스크 미착용으로 판별되면, 
                    cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                    Y = startY - 10 if startY - 10 > 10 else startY + 10
                    text = "driver {:.2f}".format((1 - prediction[0][0]))
                    cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                    n_count = 0

                else: # 마스크 착용으로 판별되면

                    n_count += 1

                    cv2.rectangle(frame, (startX,startY), (endX,endY), (0,0,255), 2)
                    Y = startY - 10 if startY - 10 > 10 else startY + 10
                    text = "other {:.2f}".format(prediction[0][0])
                    cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                   # gotojsp_theaf(login_no)   
                if n_count > 5:
                    print("도난")
                    #gotojsp_theaf(login_no)                     
                    n_count=0

        # display output
        cv2.imshow("driver other classify", frame)

        # press "Q" to stop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # release resources
webcam.release()
cv2.destroyAllWindows() 